In [ ]:
import pandas as pd
import numpy as np
import locale
import os
import re
import time
import pickle
from collections import defaultdict
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Process tabular data

Designed to process Kelvin Mu's market summaries,
- Funding in April
- Funding in March

In [ ]:
regions = ['Americas', 'North America', 'South America', 
           'Europe', 'Africa', 'Asia', 'Middle East', 
           'Oceania', 'Australia',
          ]
column_names = ['Companies', 'HQ Global Region', 'Last Financing Size', 'Total Raised', 'Description']

def process_line_fundingfirst(l, regions):
    split_string = '({region_string})'.format(region_string='|'.join(regions))    
    line_split = re.split(split_string, l, 1)
    line_split = [s.strip() for s in line_split]
    company_funding, region, description = line_split
    company_funding = company_funding.split('$')
    company, total_financing, last_financing = [s.strip() for s in company_funding]
    total_financing = locale.atoi(total_financing)
    last_financing = locale.atoi(last_financing)
    return company, region, last_financing, total_financing, description  

def process_line_regionfirst(l, regions):
    split_string = '({region_string})'.format(region_string='|'.join(regions))    
    line_split = re.split(split_string, l, 1)
    line_split = [s.strip() for s in line_split]
    company, region, remainder = line_split
    remainder = remainder.split('$')[1:]
    last_financing = locale.atoi(remainder[0])
    total_financing, description = remainder[1].split(' ', 1)
    total_financing = locale.atoi(total_financing)
    return company, region, last_financing, total_financing, description    


file_specs = [('march', 'data/march_ai_deals.txt', process_line_fundingfirst),
              ('april', 'data/april_ai_deals.txt', process_line_regionfirst),
             ]
min_funding_size = 10.0

month_order = ['march', 'april']

res = {}
funding_rounds = {}
company_attributes = pd.DataFrame()

for prefix, fname, processor in file_specs:

    lines = []

    with open(fname, 'r') as f:
        for l in f.readlines():
            lines.append(processor(l, regions))    
    #     lines = [process_line(l, regions) for l in f.readlines()]

    tmp = pd.DataFrame.from_records(lines, columns=column_names).set_index('Companies')
    res[prefix] = tmp
    
df = pd.concat(res, names=['month'])
df = df[df['Last Financing Size'] >= min_funding_size]

prev_maxrows = pd.get_option('display.max_rows')

pd.set_option('display.max_rows', 500)
display(df)
pd.set_option('display.max_rows', prev_maxrows)

df.to_csv(fname.replace('.txt', '.csv'), index=False)

company_list = sorted(df.index.unique('Companies'))

# Company attributes: Prefer most recent version of attributes
attribute_columns = ['HQ Global Region', 'Description']
company_attributes = pd.DataFrame(index=company_list, columns=attribute_columns)

for month in month_order[::-1]:
    company_attributes.update(res[month][attribute_columns])

# Company funding: Prefer the last version of total funding
total_raised_alltime = {}
for month in month_order:
    company_attributes[month] = res[month]['Last Financing Size']
    total_raised_alltime.update(res[month]['Total Raised'].to_dict())
    
company_attributes['Total Raised'] = pd.Series(total_raised_alltime)

# Get rid of companies which are shown as raising in two consecutive months, which is probably bad data
mask = company_attributes[month_order].diff(1).notnull().shift(-1, axis=1).iloc[:, :-1]
company_attributes[mask.columns] = company_attributes[mask.columns].mask(mask, np.nan)
company_attributes[month_order] = company_attributes[month_order].fillna(0)
company_attributes['Raised before 2023-03'] = company_attributes['Total Raised'] - company_attributes[month_order].sum(1)

# Programmatically get estimated Crunchbase urls:

In [ ]:
def get_crunchbase_url(company_name):
    formatted_name = re.sub('[ .:,]', '-', company_name.lower())

    headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "sec-ch-ua": "\"Not.A/Brand\";v=\"8\", \"Chromium\";v=\"114\", \"Google Chrome\";v=\"114\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "cookie": os.environ['CRUNCHBASE_COOKIE'],
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "upgrade-insecure-requests": "1",
        "authority": "www.crunchbase.com",
    }

    url = f'https://www.crunchbase.com/organization/{formatted_name}'
    print(url)


    r = requests.get(url, headers=headers)

    assert r.status_code != 403, "Robots error!"

    soup = BeautifulSoup(r.text, 'html.parser')
    content_sections = soup.find_all('h2', class_='section-title')
    
    if len(content_sections) > 0:
        return url
    else:
        return ""

company_detail_fname = 'company_specs.pkl'
    
company_list = sorted(df.index.unique('Companies'))

# If we've previously saved the company specs, go ahead and 
if os.path.exists(company_detail_fname):
    company_specs = pickle.load(open(company_detail_fname, 'rb'))
    companies_to_process = sorted(set(company_list) - set(company_specs.keys()))
else:
    company_specs = defaultdict(dict)
    companies_to_process = company_list
    
for company_name in companies_to_process:
    try:
        company_specs[company_name]['crunchbase_url'] = get_crunchbase_url(company_name)
        time.sleep(5)
    except AssertionError as e:
        if str(e) == "Robots error!":
            print("Robots Error on "+company_name)
            break
        else:
            print("Other error: " + str(e))
pickle.dump(company_specs, open(company_detail_fname, 'wb'))

company_spec_df = pd.DataFrame(company_specs).T



## Finalize and clean up

In [ ]:
full_details = company_attributes.join(company_spec_df, how='left')
full_details.to_csv('results/ai_deals_total.csv')
full_details.head()

financial_column_renamings = {m: m.capitalize() for m in month_order}
financial_column_renamings['Raised before 2023-03'] = 'Feb'
financials = full_details[financial_column_renamings.keys()].rename(financial_column_renamings, axis=1)

capitalized_month_order = [m.capitalize() for m in month_order]
full_financial_column_order = ['Feb'] + capitalized_month_order
financials = financials[full_financial_column_order]

# Exploratory data analysis

In [ ]:
# Initial descriptive analytics
for m in capitalized_month_order:
    tmp = financials[m].replace(0, np.nan).dropna()
    display(tmp.describe())
    display(tmp[tmp > 149])
    tmp.hist()
    plt.show(); plt.close()

In [ ]:

mpl.style.use('seaborn-v0_8')

lw = 1.5
linestyle='solid'

max_value_on_hist = 200
bins = range(int(min_funding_size), max_value_on_hist+1, 5)
res = pd.DataFrame(index=bins[:-1])

for m in capitalized_month_order:
    tmp = financials[m].replace(0, np.nan).dropna()
    bars, _ = np.histogram(tmp.values, bins)
    res[m] = bars

fig, ax = plt.subplots(figsize=(4, 4))

# ax = res.plot(drawstyle="steps-post", ax=ax)
ax.stairs(res['March'], label='March', edges=np.append(res.index.values, max_value_on_hist), fill=True, color='tab:blue', alpha=0.5)
ax.stairs(res['April'], label='April', edges=np.append(res.index.values, max_value_on_hist), fill=True, color='tab:green', alpha=0.5)
ax.set_ylabel('Number of startups funded')
ax.set_xlabel('Funding Round Size, $M')
plt.legend()
# ax.get_legend().remove()

In [ ]:
from matplotlib.ticker import FuncFormatter

cumulative_funding = {}

for m in capitalized_month_order:
    tmp = financials[m].sort_values()
    cumulative_funding[m] = pd.Series(tmp.values, index=tmp.values).groupby(level=0).sum()
    
fig, ax = plt.subplots(figsize=(4, 4))
pd.DataFrame(cumulative_funding).fillna(0).cumsum().plot(ax=ax)
ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_ylabel('Cumulative funding raised, all startups, $M')
ax.set_xlabel('Funding Round Size, $M')

In [ ]:
totals = financials.sum()

relative_increase = totals / totals.cumsum()

fig, ax = plt.subplots(figsize=(4, 4))
# totals.rename({'Feb':'Prior to\n2023-03'}).cumsum().plot(kind='bar')
cum_totals = totals.rename({'Feb':'Prior to\n2023-03'}).cumsum()
cum_totals /= 1000
plt.bar(cum_totals.index, cum_totals.values)

for m in capitalized_month_order:
    plt.text(m, cum_totals[m] + 1, '+ %.0f%%'%(relative_increase[m]*100))

plt.xticks(rotation=0)
ax.set_ylim((0, cum_totals.max() + 3))
ax.set_ylabel('Total raised in sector to date, $B')